In [77]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [78]:
data = pd.read_csv("liver_disease_dataset.csv")  # Replace with actual dataset path

In [79]:
data.head()

,Age,Sex,AST (SGOT),ALT (SGPT),AST:ALT Ratio,GGTP,ALP,Bilirubin Total,Bilirubin Direct,Bilirubin Indirect,Total Protein,Albumin,A:G Ratio,Condition
0,71,Female,139,245,0.567347,466,261,3.528115,0.847263,2.680853,7.385806,4.579157,1.631533,0
1,34,Female,32,200,0.160000,432,267,0.946800,0.113197,0.833603,7.424992,2.602252,0.539578,0
2,80,Male,288,110,2.618182,109,149,3.033669,0.324436,2.709233,5.060133,3.652672,2.595202,4
3,40,Female,260,114,2.280702,412,136,1.459950,0.403953,1.055996,5.500904,5.460975,136.735229,0
4,43,Male,18,259,0.069498,585,279,4.726851,0.933136,3.793715,5.946073,4.252944,2.511869,0


In [80]:
# Encode categorical column 'Sex'
label_encoder = LabelEncoder()
data['Sex'] = label_encoder.fit_transform(data['Sex'])

In [81]:
data.head()

,Age,Sex,AST (SGOT),ALT (SGPT),AST:ALT Ratio,GGTP,ALP,Bilirubin Total,Bilirubin Direct,Bilirubin Indirect,Total Protein,Albumin,A:G Ratio,Condition
0,71,0,139,245,0.567347,466,261,3.528115,0.847263,2.680853,7.385806,4.579157,1.631533,0
1,34,0,32,200,0.160000,432,267,0.946800,0.113197,0.833603,7.424992,2.602252,0.539578,0
2,80,1,288,110,2.618182,109,149,3.033669,0.324436,2.709233,5.060133,3.652672,2.595202,4
3,40,0,260,114,2.280702,412,136,1.459950,0.403953,1.055996,5.500904,5.460975,136.735229,0
4,43,1,18,259,0.069498,585,279,4.726851,0.933136,3.793715,5.946073,4.252944,2.511869,0


In [82]:
data[['GGTP', 'ALP', 'Bilirubin Total']] = np.log1p(data[['GGTP', 'ALP', 'Bilirubin Total']])

In [83]:
# Extract features and target
X = data.drop(columns=['Condition'])
y = data['Condition']

In [84]:
# Normalize feature
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Reshape input for LSTM (samples, time steps, features)
X_reshaped = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))

In [85]:
# Convert target to categorical
num_classes = len(np.unique(y))
y_categorical = to_categorical(y, num_classes=num_classes)

In [86]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y_categorical, test_size=0.2, random_state=42)

In [87]:
model = Sequential([
    LSTM(128, return_sequences=True, input_shape=(1, X.shape[1])),
    Dropout(0.3),
    LSTM(64, return_sequences=True),
    Dropout(0.3),
    LSTM(32),
    Dropout(0.3),
    Dense(16, activation='relu'),
    Dense(num_classes, activation='softmax')
])


In [88]:
from tensorflow.keras.optimizers import RMSprop
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=0.001), metrics=['accuracy'])


In [89]:
# Train model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/50
125/125 [==============================] - 4s 9ms/step - loss: 1.5274 - accuracy: 0.4058 - val_loss: 1.4734 - val_accuracy: 0.4210
Epoch 2/50
125/125 [==============================] - 0s 3ms/step - loss: 1.4846 - accuracy: 0.4075 - val_loss: 1.4747 - val_accuracy: 0.4210
Epoch 3/50
125/125 [==============================] - 0s 4ms/step - loss: 1.4872 - accuracy: 0.4075 - val_loss: 1.4742 - val_accuracy: 0.4210
Epoch 4/50
125/125 [==============================] - 0s 3ms/step - loss: 1.4844 - accuracy: 0.4075 - val_loss: 1.4752 - val_accuracy: 0.4210
Epoch 5/50
125/125 [==============================] - 0s 3ms/step - loss: 1.4853 - accuracy: 0.4075 - val_loss: 1.4745 - val_accuracy: 0.4210
Epoch 6/50
125/125 [==============================] - 0s 3ms/step - loss: 1.4845 - accuracy: 0.4075 - val_loss: 1.4747 - val_accuracy: 0.4210
Epoch 7/50
125/125 [==============================] - 0s 3ms/step - loss: 1.4840 - accuracy: 0.4075 - val_loss: 1.4769 - val_accuracy: 0.4210
Epoch 

In [90]:
# Evaluate
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

32/32 [==============================] - 0s 1ms/step - loss: 1.4801 - accuracy: 0.4210
Test Accuracy: 42.10%
